### On importe les bilbliothéques qu'on va utilisé sur ce Notebook

In [1]:
# Importing librairies
import pandas as pd
import nltk
import numpy as np
import pickle
from bs4 import BeautifulSoup
import stop_words
import re, sys
import spacy
import warnings
warnings.filterwarnings('ignore')
import pickle
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
import enchant
from enchant.checker import SpellChecker

### On importe la table question ---> tag et on la transforme sous une forme exploitable

In [2]:
### on cmmence par joindre les tables
df = pd.read_csv('quesiton_tag_100_25_03.csv')

In [3]:
df['tag'].value_counts()

notaire         76
transmission    73
patrimoine      52
droit           46
fiscalité       45
banque          38
assurance       26
agent           17
rendement       16
crowdfunding     1
Name: tag, dtype: int64

In [120]:
df.rename(columns={'message_y':'message'},inplace=True)

## On Rajoute chaque tag dans notre texte

In [122]:
top_tags = df['tag'].unique().tolist()

In [123]:
def trans(val):
    val = " ".join(str(x) for x in val)
    return val

In [124]:
index = [i+10000000 for i in range(len(top_tags))]

In [125]:
a = pd.DataFrame()
a['id'] = index
a['message'] = top_tags
a['tag'] = top_tags

In [126]:
df = pd.concat([df,a])

### On garde une phrase avec tous les tags correspond par ligne

In [128]:
dfg = df.groupby('message')['tag'].unique().reset_index()

In [130]:
dfg['Tag'] = dfg['tag'].apply(trans)

In [2]:
## dealing with message
def correct_text(x):
    chkr = enchant.checker.SpellChecker("fr_FR")
    chkr.set_text(x)
    for err in chkr:
        try:
            sug = err.suggest()[0]
            err.replace(sug)
        except : pass
    c = chkr.get_text()#returns corrected text
    return c
def tokenization(x):
    tokenizer = nltk.RegexpTokenizer(r'\w+') # Use regular expression to delete \n
    x = tokenizer.tokenize(x.lower())
    return x
# Define stopwords
sw = stop_words.get_stop_words(language='fr')
def del_stopwords(x):
    x = [t.lower() for t in re.split(" ", re.sub(r"(\W+|_|\d+)", " ", " ".join(x)))
                 if t.lower() not in sw and len(t) > 1]
    return x
lem = spacy.load('fr_core_news_md')
def lematization(x):
    x = [tok.lemma_ for tok in lem(' '.join(x))]
    return x
def preprocessing(x):
    return lematization(del_stopwords(tokenization(correct_text(x))))

In [132]:
dfg['tokens_clean_lemma'] = dfg['message'].apply(preprocessing)

In [133]:
dfg.shape

(150, 4)

In [135]:
dfg['tokens_clean_lemma'].loc[5]

['héritage',
 'soeur',
 'mèr',
 'rester',
 'succession',
 'pèr',
 'notair',
 'dire',
 'soeur',
 'mer',
 'rester',
 'normal',
 'merci',
 'pro',
 'réponse']

# on termine le preprocessing des messages on cree une nouvelle table data sur laquelle on entraine notre modéle

In [137]:
data = dfg.copy()

## on entraine notre vectorizer tfidf

In [138]:
corpus = data['tokens_clean_lemma'].apply(trans).values
vectorizer = TfidfVectorizer(ngram_range=(1, 1)) # Creating tf-idf matrix using unigram
matrix = vectorizer.fit_transform(corpus) 
matrix_ = matrix.todense()
tf_idfmatrix = pd.DataFrame(data=matrix_,columns=vectorizer.get_feature_names(), index=data.index)
tf_idfmatrix = pd.concat([data['message'],data['tokens_clean_lemma'],tf_idfmatrix, data['Tag']], ignore_index=False, axis=1)

## on définie le score MAPK 

In [139]:
def apk(actual, predicted, k=5):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=5):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

## On définit la Cross-Validation

In [140]:
def cross_val(X,y,test_size,k,n_splits=5):
    cv = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=0)
    scores = []
    for indice_train, indice_test in cv.split(X):
        X_train = X[indice_train]
        X_test = X[indice_test]
        y_train = y[indice_train]
        y_test = y[indice_test]
        #     Target binarizing
        mlb = MultiLabelBinarizer(classes=df['tag'].unique())
        mlb.fit([w.split() for w in y_train])
        y_train_mlb = mlb.transform([w.split() for w in y_train])
        y_test_mlb = mlb.transform([w.split() for w in y_test])
        m_NB = OneVsRestClassifier(MultinomialNB(alpha=0.001))
        m_NB.fit(X_train[:,2:], y_train_mlb)
        y_pred_NB = mlb.classes_[np.argsort(m_NB.predict_proba(X_test[:,2:]))][:, :-(200+1): -1]
        c = mlb.inverse_transform(y_test_mlb)
        y_ij = []
        for elem in c:
            a = []
            for i in range(len(elem)):
                a.append(elem[i])
            y_ij.append(a)
        scores.append(mapk(y_ij, y_pred_NB, k=k))
    print(indice_train.shape,indice_test.shape)
    print('scores :',scores)
    print('mean',np.mean(scores))

## On teste avec plusieurs paramétre k à chaque fois on change le test_size (20% , 30% , 50%)

In [141]:
X = tf_idfmatrix.iloc[:, :-1].values
y = tf_idfmatrix['Tag']
for k in range(1,8):
    cross_val(X,y,0.2,k)

(120,) (30,)
scores : [0.7, 0.6333333333333333, 0.8, 0.6666666666666666, 0.7]
mean 0.7
(120,) (30,)
scores : [0.5583333333333333, 0.65, 0.6833333333333333, 0.5666666666666667, 0.6583333333333333]
mean 0.6233333333333333
(120,) (30,)
scores : [0.6101851851851852, 0.6583333333333333, 0.6518518518518519, 0.5703703703703702, 0.6398148148148147]
mean 0.6261111111111111
(120,) (30,)
scores : [0.639351851851852, 0.6641203703703703, 0.6643518518518517, 0.6226851851851851, 0.6814814814814815]
mean 0.6543981481481481
(120,) (30,)
scores : [0.6427962962962963, 0.6690925925925927, 0.6853518518518519, 0.649351851851852, 0.6842870370370371]
mean 0.6661759259259259
(120,) (30,)
scores : [0.6799629629629628, 0.6983518518518519, 0.7168333333333333, 0.673425925925926, 0.7035462962962965]
mean 0.6944240740740741
(120,) (30,)
scores : [0.7037724867724867, 0.7142248677248677, 0.7444523809523811, 0.6861243386243387, 0.7264034391534392]
mean 0.7149955026455027


In [83]:
X = tf_idfmatrix.iloc[:, :-1].values
y = tf_idfmatrix['Tag']
for k in range(1,8):
    print("K " , k)
    cross_val(X,y,0.3,k)

K  1
(105,) (45,)
scores : [0.6888888888888889, 0.6666666666666666, 0.7333333333333333, 0.7555555555555555, 0.6666666666666666]
mean 0.7022222222222222
K  2
(105,) (45,)
scores : [0.5888888888888889, 0.6555555555555556, 0.65, 0.6555555555555556, 0.5722222222222222]
mean 0.6244444444444444
K  3
(105,) (45,)
scores : [0.5962962962962962, 0.625925925925926, 0.6425925925925925, 0.6691358024691358, 0.5604938271604938]
mean 0.6188888888888888
K  4
(105,) (45,)
scores : [0.6453703703703704, 0.6277777777777778, 0.6594135802469135, 0.6748456790123457, 0.6087962962962962]
mean 0.6432407407407408
K  5
(105,) (45,)
scores : [0.6352222222222221, 0.6532592592592593, 0.6735246913580246, 0.7104012345679013, 0.6401851851851851]
mean 0.6625185185185185
K  6
(105,) (45,)
scores : [0.6698765432098764, 0.6730123456790124, 0.6992037037037035, 0.733858024691358, 0.6661111111111111]
mean 0.6884123456790123
K  7
(105,) (45,)
scores : [0.691463844797178, 0.693541446208113, 0.7292566137566138, 0.7391490299823632

In [45]:
X = tf_idfmatrix.iloc[:, :-1].values
y = tf_idfmatrix['Tag']
for k in range(1,8):
    cross_val(X,y,0.5,k)

(75,) (75,)
scores : [0.7333333333333333, 0.72, 0.5866666666666667, 0.6666666666666666, 0.6666666666666666]
mean 0.6746666666666666
(75,) (75,)
scores : [0.66, 0.7066666666666667, 0.5466666666666666, 0.61, 0.61]
mean 0.6266666666666667
(75,) (75,)
scores : [0.6581481481481481, 0.7174074074074074, 0.5525925925925925, 0.6381481481481482, 0.6303703703703704]
mean 0.6393333333333333
(75,) (75,)
scores : [0.6758333333333333, 0.7210185185185185, 0.5949074074074074, 0.6390740740740741, 0.6604629629629629]
mean 0.6582592592592593
(75,) (75,)
scores : [0.7006777777777778, 0.7279074074074073, 0.6217518518518518, 0.6696851851851852, 0.6857851851851852]
mean 0.6811614814814815
(75,) (75,)
scores : [0.7200481481481482, 0.7496851851851852, 0.653788888888889, 0.6891666666666666, 0.7004518518518519]
mean 0.7026281481481481
(75,) (75,)
scores : [0.7342703703703705, 0.7630185185185184, 0.6680111111111111, 0.7028174603174603, 0.7074359788359788]
mean 0.7151106878306879


## entrainement du modéle

In [46]:
## on entraine notre modéle sur la partie contenant les tags dans ses textes
tr_tags = tf_idfmatrix[tf_idfmatrix['message'].isin(top_tags)]
tf_idfmatrix1 = tf_idfmatrix[~(tf_idfmatrix['message'].isin(top_tags))]

In [47]:
# Splitting data in training and test set
X = tf_idfmatrix1.iloc[:, :-1]
y = tf_idfmatrix1['Tag']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3)
X_train = pd.concat([X_train,tr_tags.iloc[:, :-1]])
y_train = pd.concat([y_train,tr_tags['Tag']])
X_train = X_train.values
X_test = X_test.values
#     Target binarizing
mlb = MultiLabelBinarizer(classes=df['tag'].unique())
mlb.fit([w.split() for w in y_train])
y_train_mlb = mlb.transform([w.split() for w in y_train])
y_test_mlb = mlb.transform([w.split() for w in y_test])
print('test size : ', X_test.shape[0])
print('train size : ', y_train_mlb.shape[0])

test size :  42
train size :  108


In [48]:
m_NB = OneVsRestClassifier(MultinomialNB(alpha=0.001))
m_NB.fit(X_train[:,2:], y_train_mlb)
y_pred_NB = mlb.classes_[np.argsort(m_NB.predict_proba(X_test[:,2:]))][:, :-(200+1): -1]
c = mlb.inverse_transform(y_test_mlb)
y_ij = []
print(X_train.shape,X_test.shape)
for elem in c:
    a = []
    for i in range(len(elem)):
        a.append(elem[i])
    y_ij.append(a)
for k in range(1,8):
    print(mapk(y_ij, y_pred_NB, k=k))

(108, 1980) (42, 1980)
0.7619047619047619
0.6785714285714286
0.6402116402116401
0.6755952380952381
0.7077579365079365
0.7302447089947091
0.7420360922146637


In [72]:
## on sauvegarde les outils qu'on a entrainer

import pickle

pickle_out = open("model_supervised_fr.pkl","wb")
pickle.dump(m_NB, pickle_out)
pickle_out.close()


pickle_out = open("binarizer_supervised_fr.pkl","wb")
pickle.dump(mlb, pickle_out)
pickle_out.close()


pickle_out = open("vectorizer_tfidf_fr.pkl","wb")
pickle.dump(vectorizer, pickle_out)
pickle_out.close()


### on affiche la table avec les questions de la partie du test ainsi que les tags prédits

In [84]:
scores = m_NB.predict_proba(X_test[:,2:])
scores.sort()
scores = scores[:,:-200:-1]
clas = y_pred_NB

In [85]:
clas_proba = []
for i in range(clas.shape[0]):
    doc = []
    for j in range(clas.shape[1]):
        doc.append((clas[i][j],scores[i][j]))
    clas_proba.append(doc)

In [86]:
final = pd.DataFrame(X_test[:,:2]).reset_index()

In [87]:
pred = pd.DataFrame(clas_proba).reset_index()

In [88]:
test_tag = final.merge(pred,how='inner',on='index')

In [90]:
test_tag['true'] = y_test.tolist()

## tager un nouveau texte

In [91]:
pickle_in = open("vectorizer_tfidf_fr.pkl","rb")
vv = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open("model_supervised_fr.pkl","rb")
mm = pickle.load(pickle_in)
pickle_in.close()

In [92]:
def predict(text):
    d = pd.DataFrame({'message':[text]})
    d['tokens_clean_lemma'] = d['message'].apply(preprocessing)
    corpus1 = d['tokens_clean_lemma'].apply(trans).values
    matrix = vv.transform(corpus1) 
    matrix_ = matrix.todense()
    X = pd.DataFrame(data=matrix_,columns=vv.get_feature_names(), index=d.index)
    y_pred_NB = mlb.classes_[np.argsort(mm.predict_proba(X))][:, :-(200+1): -1]
    scores = mm.predict_proba(X)
    scores.sort()
    scores = scores[:,:-200:-1]
    clas = y_pred_NB
    clas_proba = []
    for i in range(clas.shape[0]):
        doc = []
        for j in range(clas.shape[1]):
            doc.append((clas[i][j],scores[i][j]))
        clas_proba.append(doc)
    return list(y_pred_NB[0][:5]),clas_proba[0]

In [93]:
## ici on éfinit une méthode pour tager les questions
def tager(clas_proba,threshold = 0.6,nbr_tag=5):
    count = 0
    tagf = []
    score = 0
    for tag in clas_proba:
        if tag[1]>threshold:
            tagf.append(tag)
            count += 1
        else :
            while score<threshold:
                score += tag[1]
                tagf.append(tag)
                break
    if len(tagf)>nbr_tag:
        return tagf[:nbr_tag]
    elif count == 0:
        return 'pas de TAG'
    else :
        return tagf 

In [144]:
tager(predict("je veux louer une maison")[1],threshold=0.8)

[('patrimoine', 0.8188278421001546),
 ('notaire', 0.6458554269156233),
 ('transmission', 0.5501330170872044)]